In [1]:
import pandas as pd
import requests
import os

In [2]:
os.chdir('D:/jean_/OneDrive/Profissional/Analista de Dados/Case técnico/Documentos')

diretorio_atual = os.getcwd()

### Download dos arquivos

In [3]:
meses = range(1,13)

url_base = 'https://ftp.ibge.gov.br/Comercio_e_Servicos/Pesquisa_Mensal_de_Comercio/Tabelas/2021/'

In [4]:
for mes in meses:
    download = requests.get(url_base + f'pmc_2021{mes:02d}_03.xls')
    open(f'pmc_2021{mes:02d}_03.xls', 'wb').write(download.content)

## Análise 1

### Transformando todos arquivos em Dataframes

In [5]:
df = {}

for mes in meses:
    df[mes] = pd.read_excel(f'pmc_2021{mes:02d}_03.xls', skiprows = 3, header = 1)

### Tratamento

In [6]:
for i in range(1,13):
    df[i].drop(df[i].tail(5).index, inplace = True)
    df[i].rename(columns = {'Unnamed: 3': 'Mês/Mês anterior', 'Unnamed: 6': 'Mensal', 'Unnamed: 9': 'Acumulado no ano'}, inplace = True)
    df[i].drop(columns = {'Mês/Mês anterior (1)', 'Unnamed: 2', 'Mensal (2)', 'Unnamed: 5', 'Acumulado no ano (3)', 'Unnamed: 8',
           'Últimos 12 meses (4)', 'Unnamed: 11', 'Unnamed: 12'}, inplace = True)
    df[i].replace(df[i].iloc[0, 3], df[i].iloc[0, 3][-3:].capitalize(), inplace = True)
    df[i]['Mês referência'] = str(i)
    df[i].drop(0, inplace = True)

### Carregando em um Dataframe

In [7]:
df_final = pd.concat([df[i] for i in range (1,13)], ignore_index = True)

### Alterando o tipo da coluna para 'Date'

In [8]:
from datetime import date

data = date(2021, 1, 1)
ano = str(data.year)
dia = str(data.day)

df_final['Mês referência'] = df_final['Mês referência'].map(lambda x: ano + '-' + x + '-' + dia)
df_final['Mês referência'] = pd.to_datetime(df_final['Mês referência'])

### Exportando Excel

In [9]:
df_final.to_excel('case_tecnico.xlsx', sheet_name = 'case_tecnico')

## Análise 2

In [49]:
df_2 = pd.read_csv('Transferencias-da-Uniao---Total-por-Estado.csv', sep = ';', encoding = 'ISO-8859-1', header = None)

In [46]:
df_2.head(5)

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,Transferencias da União para o Estado do Acre ...,jan/11,fev/11,mar/11,abr/11,mai/11,jun/11,jul/11,ago/11,set/11,...,fev/22,mar/22,abr/22,mai/22,jun/22,jul/22,ago/22,set/22,out/22,nov/22
1,1. Transferências Obrigatórias,"189,1214","201,4436","135,9553","178,0381","200,5328","180,8686","160,9362","161,8701","132,6303",...,"615,7978","404,2287","446,8937","490,3168","461,5978","405,1630","456,9430","384,0283","402,7929","505,4247"
2,1.1 - Transferências Obrigatórias Constitucionais,"180,7493","191,7863","125,9610","168,8363","190,4786","172,0476","149,9365","151,1455","120,9181",...,"596,2480","357,4618","427,9253","469,7458","442,4405","386,6082","438,2010","365,2417","382,8102","483,6152"
3,1.2 - Transferências Obrigatórias Legais,"8,3721","9,6574","9,9942","9,2018","10,0542","8,8210","10,9997","10,7246","11,7122",...,"19,5498","46,7669","18,9685","20,5710","19,1573","18,5549","18,7420","18,7866","19,9827","21,8095"
4,2. Transferências Discricionárias,"17,8065","3,5447","44,8994","4,5810","6,2439","4,0471","48,1849","11,5312","10,4542",...,"3,9944","2,3482","0,5188","15,5676","31,0142","12,5379","2,0392","1,9873","18,9435","0,9654"


In [50]:
df_2 = df_2[df_2[0].str.contains('1.1', regex = True) == False]
df_2 = df_2[df_2[0].str.contains('1.2', regex = True) == False]
df_2 = df_2[df_2[0].str.contains('2.1', regex = True) == False]
df_2 = df_2[df_2[0].str.contains('2.2', regex = True) == False]
df_2 = df_2[df_2[0].str.contains('2.3', regex = True) == False]

In [59]:
df_2

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,Transferencias da União para o Estado do Acre ...,jan/11,fev/11,mar/11,abr/11,mai/11,jun/11,jul/11,ago/11,set/11,...,fev/22,mar/22,abr/22,mai/22,jun/22,jul/22,ago/22,set/22,out/22,nov/22
1,1. Transferências Obrigatórias,"189,1214","201,4436","135,9553","178,0381","200,5328","180,8686","160,9362","161,8701","132,6303",...,"615,7978","404,2287","446,8937","490,3168","461,5978","405,1630","456,9430","384,0283","402,7929","505,4247"
4,2. Transferências Discricionárias,"17,8065","3,5447","44,8994","4,5810","6,2439","4,0471","48,1849","11,5312","10,4542",...,"3,9944","2,3482","0,5188","15,5676","31,0142","12,5379","2,0392","1,9873","18,9435","0,9654"
8,Transferencias da União para o Estado de Alago...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1. Transferências Obrigatórias,"237,8609","251,1571","170,1652","235,3347","247,9944","227,2455","202,8828","197,6508","156,8256",...,"707,6910","527,1807","532,9401","575,5303","546,0706","481,0447","538,9233","458,5783","481,6421","594,9289"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,1. Transferências Obrigatórias,"794,5596","769,1113","694,1588","670,8738","737,0317","644,9713","776,1417","646,3980","653,0527",...,"1817,7202","1790,6894","1274,0293","2044,4853","1271,8361","1258,6287","2016,9904","1233,5901","2245,2973","2309,6370"
204,2. Transferências Discricionárias,"92,7519","55,1478","32,7914","119,0796","26,9960","167,0252","78,1544","18,8158","20,5621",...,"11,8147","36,7858","11,4637","17,3014","441,9061","16,6300","4,6673","11,7541","254,8447","8,2293"
208,Transferencias da União para o Estado de Tocan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,1. Transferências Obrigatórias,"250,5217","262,2362","179,7820","229,0071","258,4910","231,4555","211,8705","207,3192","169,5429",...,"684,4118","490,3283","515,4251","576,5257","532,7618","453,7582","509,3596","429,0186","460,4950","557,3937"


In [58]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 212
Columns: 144 entries, 0 to 143
dtypes: object(144)
memory usage: 91.8+ KB


In [55]:
df_2.to_excel('case_tecnico_2.xlsx', sheet_name = 'case_tecnico_2')